In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(5000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

/home/luca/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9973280296027781, 0.9970176223359105)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9995569078933944, 0.9997519827416904)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NameSurname,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,Error,Error,Region,Municipality,...,CountryCode,Name,SurnameName,NaN,String,EMail,PhoneNumber,Date,NaN,Boolean
1,Error,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,NaN,Name,Error,NaN,String,EMail,Error,Date,BiologicalSex,NaN
2,ItalianFiscalCode,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,SurnameName,Surname,String,EMail,Error,Date,BiologicalSex,Boolean
3,NaN,CadastreCode,NaN,NaN,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,Name,NameSurname,Surname,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,NaN,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Error,...,CountryCode,Name,NameSurname,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
5,ItalianVAT,NaN,NaN,NaN,NaN,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,Name,Error,Surname,String,EMail,PhoneNumber,NaN,NaN,NaN
6,NaN,Error,Document,NaN,Plate,NaN,ItalianZIPCode,Error,Region,Municipality,...,CountryCode,Name,Error,NaN,String,EMail,PhoneNumber,Error,BiologicalSex,Boolean


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NameSurname,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,Error,Error,Region,Municipality,...,CountryCode,Name,NameSurname,NaN,String,EMail,PhoneNumber,Date,NaN,Boolean
1,Error,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,NaN,...,NaN,Name,Error,NaN,String,EMail,Error,Date,BiologicalSex,NaN
2,ItalianFiscalCode,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Error,Surname,String,EMail,Error,Date,BiologicalSex,Boolean
3,NaN,CadastreCode,NaN,NaN,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,Name,NameSurname,Surname,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,NaN,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Error,...,CountryCode,Name,NameSurname,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
5,ItalianVAT,NaN,NaN,NaN,NaN,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,Name,Error,Surname,String,EMail,PhoneNumber,NaN,NaN,NaN
6,NaN,Error,Document,NaN,Plate,NaN,ItalianZIPCode,Error,Region,Municipality,...,CountryCode,Name,Error,NaN,String,EMail,PhoneNumber,Error,BiologicalSex,Boolean


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Integer', 'ItalianZIPCode'): 32,
         ('Error', 'ItalianFiscalCode'): 26,
         ('Error', 'NameSurname'): 85,
         ('CountryCode', 'NaN'): 58,
         ('Error', 'CountryCode'): 32,
         ('Error', 'Document'): 128,
         ('NameSurname', 'SurnameName'): 87,
         ('SurnameName', 'NameSurname'): 146,
         ('Error', 'Country'): 12,
         ('PhoneNumber', 'Error'): 17,
         ('Error', 'SurnameName'): 92,
         ('Document', 'Error'): 60,
         ('Error', 'ProvinceCode'): 41,
         ('NameSurname', 'Error'): 144,
         ('SurnameName', 'String'): 8,
         ('NameSurname', 'String'): 13,
         ('Year', 'Error'): 5,
         ('String', 'Error'): 45,
         ('String', 'EMail'): 2,
         ('NaN', 'CountryCode'): 77,
         ('ItalianZIPCode', 'Error'): 24,
         ('SurnameName', 'Error'): 165,
         ('Error', 'Region'): 7,
         ('PhoneNumber', 'String'): 6,
         ('Error', 'ItalianVAT'): 23,
         ('ProvinceCode', 'Error'